In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-fongxwyi/gym-walk_e90a1aa4ef7c4375b76526840aae5416
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-fongxwyi/gym-walk_e90a1aa4ef7c4375b76526840aae5416
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=8a523f26b7778d8647663f994463e8183727eb63445f70dd9ba406fd7a8c61a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-6i_d80c9/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gymnasium as gym
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [ ]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.action_space.seed(123) # Use action_space.seed for gymnasium
    results = []
    for _ in range(n_episodes):
        state, _ = env.reset() # Gymnasium's reset returns a tuple
        done, steps = False, 0
        while not done and steps < max_steps:
            state, _, done, _, h = env.step(pi(state)) # Gymnasium's step returns 5 values
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.action_space.seed(123) # Use action_space.seed for gymnasium
    results = []
    for _ in range(n_episodes):
        state, _ = env.reset() # Gymnasium's reset returns a tuple
        done, steps = False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _, _ = env.step(pi(state)) # Gymnasium's step returns 5 values
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [ ]:
envdesc  = ['SFFF','FHFH','FFHF', 'FFGH']
env = gym.make('FrozenLake-v1',desc=envdesc)
init_state, _ = env.reset()
goal_state = 14
P = env.env.P

In [ ]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [ ]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
        for s in range(len(P)):
            for a in range(len(P[s])):
                for prob, next_state, reward, done in P[s][a]:
                    Q[s][a] += prob * (reward+gamma*V[next_state]*(not done))
        if np.max(np.abs(V-np.max(Q, axis=1))) < theta:
            break
        V = np.max(Q, axis=1)
    pi= lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

    return V, pi

In [ ]:
print("Name:            ALAN ZION H      Register Number:    212223240004   ")
print('Optimal policy and state-value function (VI):')
print_policy(pi_best_v, P)



Name:            ALAN ZION H      Register Number:    212223240004   
Optimal policy and state-value function (VI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      < | 09      < |           | 11      < |
| 12      v | 13      v |           |           |


In [ ]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=goal_state)*100,
    mean_return(env, pi_best_v)))


Reaches goal 99.00%. Obtains an average undiscounted return of 0.9900.


In [ ]:
print_state_value_function(V_best_v, P, prec=4)


State-value function:
| 00 0.7798 | 01 0.7176 | 02 0.6772 | 03 0.6572 |
| 04 0.8034 |           | 06 0.2235 |           |
| 08 0.8514 | 09 0.5955 |           | 11    0.0 |
| 12 0.9251 | 13 0.9532 |           |           |
